In [1]:
from lightgbm import LGBMClassifier

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import GridSearchCV
import optuna
from optuna.trial import Trial
from optuna.samplers import TPESampler
import warnings
warnings.filterwarnings('ignore')

from typing import List, Any, Tuple
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Data Loading
X_model = pd.read_csv('X_model.csv')
Y_model = pd.read_csv('Y_model.csv')
scaler = MinMaxScaler(feature_range=(0,1))

In [3]:
# Filter outliers by "entire"
# Ref: https://machinelearningmastery.com/how-to-use-statistics-to-identify-outliers-in-data/
print("Filtering outliers...")
df_base = pd.concat([X_model, Y_model], axis=1)
df_processed = pd.DataFrame(data=df_base, columns=['business'])
df_processed['cEntire'] = df_base.filter(regex="c" + r"2022[0-9]*", axis=1).fillna(0).sum(axis=1)
df_processed['tEntire'] = df_base.filter(regex="t" + r"2022[0-9]*", axis=1).fillna(0).sum(axis=1)
df_processed['sEntire'] = df_base.filter(regex="s" + r"2022[0-9]*", axis=1).fillna(0).sum(axis=1)

outliers = []

def collect_outliers(business: int, key: str):
    df_target = df_processed[df_processed['business'] == business]
    q1, q3 = df_target[key].quantile([0.25, 0.75])
    iqr = q3 - q1
    cutoff = iqr * 1.5
    lower, upper = q1 - cutoff, q3 + cutoff
    _outliers = df_target[(df_target[key] < lower) | (df_target[key] > upper)].index.tolist()
    print(len(_outliers))
    outliers.extend(_outliers)

collect_outliers(0, 'cEntire')
collect_outliers(1, 'cEntire')
collect_outliers(0, 'tEntire')
collect_outliers(1, 'tEntire')
collect_outliers(0, 'sEntire')
collect_outliers(1, 'sEntire')

outliers = list(set(outliers))

# Filter outliers from df
def filter_outliers_from_df(df: pd.DataFrame, outliers):
    return df.drop(outliers)

X_model = filter_outliers_from_df(X_model, outliers)
Y_model = filter_outliers_from_df(Y_model, outliers)

# Define scaler
# print("Defining scaler...")
# scaler = MinMaxScaler(feature_range=(0,1))

# Define preprocessors
print("Defining preprocessors...")
def column(colnames: List[str]):
    def _column(X: pd.DataFrame):
        X = X.fillna(0)
        return [
            [colname, X[colname].values] for colname in colnames
        ]
    return _column

def rangesum(
    name:str, 
    regex: str, 
    prefixes: str, 
    dist: np.ndarray
):
    def _rangesum(X: pd.DataFrame):
        X = X.fillna(0)
        return [
            [
                prefix + name, 
                X.filter(regex=(prefix + regex), axis=1).values.dot(dist)
            ] for prefix in prefixes
        ]
    return _rangesum

def rangesum_from_list(
    name: str, 
    namelist: List[str], 
    prefix: str,
    dist: np.ndarray,    
):
    def _rangesum_from_list(X: pd.DataFrame):
        X = X.fillna(0)
        return [
            [
                prefix + name, 
                X[namelist].values.dot(dist)
            ]
        ]
    return _rangesum_from_list

def _fillna(X: np.ndarray) -> np.ndarray:
    return np.nan_to_num(X, copy=True, nan=0)

def array_divide(
    numerator: List[Tuple[str, np.ndarray]], 
    denominator: List[Tuple[str, np.ndarray]]
) -> List[Any]:
    assert len(numerator) == len(denominator)
    return [
        [
            "r" + numerator_colname, 
            _fillna(np.divide(numerator_col, denominator_col))
        ] for [numerator_colname, numerator_col], [_, denominator_col] in zip(numerator, denominator)
    ]

def one_hot_encode(column: str) -> pd.DataFrame:
    def _one_hot_encode(X: pd.DataFrame):
        X = X.fillna(0)
        df_dummies = pd.get_dummies(X[column], prefix=column)
        return [
            [colname, df_dummies[colname].values] for colname in df_dummies.columns
        ]
    return _one_hot_encode

def preprocess(X: pd.DataFrame, processors: List[Any]) -> pd.DataFrame:
    X_new = pd.DataFrame()

    for processor in processors:
        for colname, col in processor if type(processor) == type([]) else processor(X):
            X_new[colname] = col

    X_new = X_new.fillna(0)

#     X_new = pd.DataFrame(scaler.fit_transform(X_new), columns=X_new.columns)

    return X_new

def equal_dist(length: int) -> np.ndarray:
    return np.ones(length)

def linear_dist(length: int) -> np.ndarray:
    return np.arange(start=0, stop=1, step=1/length)

def triangle_dist(length: int) -> np.ndarray:
    return np.concatenate(
        [
            np.arange(start=0, stop=1, step=1/length),
            np.arange(start=1, stop=0, step=-1/length)
        ]
    )

Filtering outliers...
62317
3595
85519
3461
63063
3548
Defining preprocessors...


In [4]:
print("Data preprocessing...")
dist_GIT = rangesum(
    'GIT', 
    r"202205[0-9]{2}", 
    "cts", 
    equal_dist(31)
)(X_model)
dist_VAT = rangesum(
    'VAT', 
    r"20220[17](?:[01][0-9]|2[0-5])", 
    "ts", 
    np.concatenate((equal_dist(25), equal_dist(25)))
)(X_model)
entire_days = 31 + 29 + 31 + 30 + 31 + 30 + 31 + 25
entire = rangesum(
    'Entire', 
    r"2022[0-9]{4}", 
    "cts", 
    equal_dist(entire_days)
)(X_model)

age_code = np.array(X_model['age_code'])
gender_code = np.array(X_model['gender'])
region_code = np.array(X_model['region_code'])
cat_Featrues = []
cat_Featrues.append(['gender',age_code])
cat_Featrues.append(['age_code',age_code])
cat_Featrues.append(['region_code',region_code])

X_processed = preprocess(
    X_model, 
    [
        cat_Featrues,
        dist_GIT,
        dist_VAT,
        entire,
        # array_divide(dist_GIT, entire), # rel_GIT
        # array_divide(dist_VAT, entire[1:]), # rel_VAT
    ]
)
X_processed.head(3)

Data preprocessing...


,gender,age_code,region_code,cGIT,tGIT,sGIT,tVAT,sVAT,cEntire,tEntire,sEntire
0,13,13,7,0.0,0.0,0.0,0.0,0.0,1.0,1.0,93.0
1,5,5,1,2.0,0.0,17.0,0.0,185.0,39.0,0.0,790.0
2,6,6,2,6.0,3.0,2253.0,0.0,1712.0,26.0,3.0,5119.0


In [5]:
print("Preparing for hyperparameter tuning...")
def _construct_and_cross_validate(**kwargs):

    lgbm = LGBMClassifier(
        task = "train",
        objective = "binary", #cross-entropy
        metric = "auc",
        tree_learner = "data",
        random_state=100,
        categorical_feature = [0,1,2],
        class_weight={0: 1, 1: 14.291397},
        n_estimators=kwargs['n_estimators'],
        # to deal with overfitting, very important param
        max_depth=kwargs['max_depth'],
        learning_rate=kwargs['learning_rate'],
        num_leaves=kwargs['num_leaves'],
        min_data_in_leaf=kwargs['min_data_in_leaf'],
        #if max_bin becomes small, the accuracy goes up
        max_bin=kwargs['max_bin'],
        lambda_l1=kwargs['lambda_l1'],
        lambda_l2=kwargs['lambda_l2'],
        # to deal with overfitting
        min_child_weight=kwargs['min_child_weight'],
        #for bagging imbalanced
        bagging_fraction=kwargs['bagging_fraction'],
        pos_bagging_fraction=kwargs['pos_bagging_fraction'],
        neg_bagging_fraction=kwargs['neg_bagging_fraction'],
    )
    #cross validation K=5
    scores = cross_val_score(
        lgbm, 
        X_processed, 
        Y_model, 
        cv=StratifiedKFold(n_splits=5, shuffle=True),
        scoring="roc_auc"
    )
    return scores

# Task: Hyperparameter tuning with Optuna
def objective(trial: Trial):
    # Construct a DecisionTreeClassifier object
    scores = _construct_and_cross_validate(
        n_estimators=trial.suggest_int('n_estimators',100,500),
        # to deal with overfitting, very important param
        max_depth = trial.suggest_int('max_depth',10,20),
        learning_rate = trial.suggest_float('learning_rate',0.02,0.1),
        num_leaves = trial.suggest_int('num_leaves',500,1000),
        min_data_in_leaf = trial.suggest_int('min_data_in_leaf',100,1000),
        #if max_bin becomes small, the accuracy goes up
        max_bin = trial.suggest_int('max_bin',255,350),
        lambda_l1 = trial.suggest_loguniform('lambda_l1', 1e-3, 10.0),
        lambda_l2 = trial.suggest_loguniform('lambda_l2', 1e-3, 10.0),
        # to deal with overfitting
        min_child_weight = trial.suggest_int('min_child_weight', 1, 10),
        #for bagging imbalanced
        bagging_fraction = trial.suggest_float('bagging_fraction', 0,1),
        pos_bagging_fraction = trial.suggest_float('pos_bagging_fraction', 0,1),
        neg_bagging_fraction = trial.suggest_float('neg_bagging_fraction', 0,1),
    )

    return scores.mean()

print("Hyperparameter tuning started...")
optuna.logging.set_verbosity(optuna.logging.WARNING)
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

# Print the best parameters
print("Best params")
print(study.best_params)

Preparing for hyperparameter tuning...
Hyperparameter tuning started...
[LightGBM] [Warning] min_data_in_leaf is set=997, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=997
[LightGBM] [Warning] lambda_l1 is set=4.233089650027591, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.233089650027591
[LightGBM] [Warning] bagging_fraction is set=0.7827500028434438, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7827500028434438
[LightGBM] [Warning] lambda_l2 is set=2.6144211987034023, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.6144211987034023
[LightGBM] [Warning] min_data_in_leaf is set=997, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=997
[LightGBM] [Warning] lambda_l1 is set=4.233089650027591, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.233089650027591
[LightGBM] [Warning] bagging_fraction is set=0.7827500028434438, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7827500

[LightGBM] [Warning] min_data_in_leaf is set=606, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=606
[LightGBM] [Warning] lambda_l1 is set=0.005311789948793842, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.005311789948793842
[LightGBM] [Warning] bagging_fraction is set=0.2079624004446442, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2079624004446442
[LightGBM] [Warning] lambda_l2 is set=0.25178529434855546, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.25178529434855546
[LightGBM] [Warning] min_data_in_leaf is set=606, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=606
[LightGBM] [Warning] lambda_l1 is set=0.005311789948793842, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.005311789948793842
[LightGBM] [Warning] bagging_fraction is set=0.2079624004446442, subsample=1.0 will be ignored. Current value: bagging_fraction=0.2079624004446442
[LightGBM] [Warning] lambda_l2 is set=0.25178529

[LightGBM] [Warning] min_data_in_leaf is set=186, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=186
[LightGBM] [Warning] lambda_l1 is set=3.301627094404139, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.301627094404139
[LightGBM] [Warning] bagging_fraction is set=0.06940310839693242, subsample=1.0 will be ignored. Current value: bagging_fraction=0.06940310839693242
[LightGBM] [Warning] lambda_l2 is set=0.014115565339048352, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.014115565339048352
[LightGBM] [Warning] min_data_in_leaf is set=186, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=186
[LightGBM] [Warning] lambda_l1 is set=3.301627094404139, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.301627094404139
[LightGBM] [Warning] bagging_fraction is set=0.06940310839693242, subsample=1.0 will be ignored. Current value: bagging_fraction=0.06940310839693242
[LightGBM] [Warning] lambda_l2 is set=0.01411556533904

[LightGBM] [Warning] min_data_in_leaf is set=226, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=226
[LightGBM] [Warning] lambda_l1 is set=0.007325297603701949, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007325297603701949
[LightGBM] [Warning] bagging_fraction is set=0.8518198671707826, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8518198671707826
[LightGBM] [Warning] lambda_l2 is set=2.379999377117308, reg_lambda=0.0 will be ignored. Current value: lambda_l2=2.379999377117308
[LightGBM] [Warning] min_data_in_leaf is set=226, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=226
[LightGBM] [Warning] lambda_l1 is set=0.007325297603701949, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.007325297603701949
[LightGBM] [Warning] bagging_fraction is set=0.8518198671707826, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8518198671707826
[LightGBM] [Warning] lambda_l2 is set=2.379999377117

In [6]:
print("Finalizing model...")
scores = _construct_and_cross_validate(
    n_estimators=study.best_params['n_estimators'],
    # to deal with overfitting, very important param
    max_depth=study.best_params['max_depth'],
    learning_rate=study.best_params['learning_rate'],
    num_leaves=study.best_params['num_leaves'],
    min_data_in_leaf=study.best_params['min_data_in_leaf'],
    #if max_bin becomes small, the accuracy goes up
    max_bin=study.best_params['max_bin'],
    lambda_l1=study.best_params['lambda_l1'],
    lambda_l2=study.best_params['lambda_l2'],
    # to deal with overfitting
    min_child_weight=study.best_params['min_child_weight'],
    #for bagging imbalanced
    bagging_fraction=study.best_params['bagging_fraction'],
    pos_bagging_fraction=study.best_params['pos_bagging_fraction'],
    neg_bagging_fraction=study.best_params['neg_bagging_fraction'],
)

print("Average ROC AUC Score", np.mean(scores))
print("Standard Deviation of ROC AUC Score", np.std(scores))


Finalizing model...
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is 

In [7]:
X_exam = pd.read_csv('X_exam.csv')

In [8]:
print("Data preprocessing...")
dist_GIT_exam = rangesum(
    'GIT', 
    r"202205[0-9]{2}", 
    "cts", 
    equal_dist(31)
)(X_exam)
dist_VAT_exam = rangesum(
    'VAT', 
    r"20220[17](?:[01][0-9]|2[0-5])", 
    "ts", 
    np.concatenate((equal_dist(25), equal_dist(25)))
)(X_exam)
entire_days = 31 + 29 + 31 + 30 + 31 + 30 + 31 + 25
entire_exam = rangesum(
    'Entire', 
    r"2022[0-9]{4}", 
    "cts", 
    equal_dist(entire_days)
)(X_exam)

age_code_exam = np.array(X_exam['age_code'])
gender_code_exam = np.array(X_exam['gender'])
region_code_exam = np.array(X_exam['region_code'])
cat_Featrues_exam = []
cat_Featrues_exam.append(['gender',age_code_exam])
cat_Featrues_exam.append(['age_code',age_code_exam])
cat_Featrues_exam.append(['region_code',region_code_exam])

X_exam_processed = preprocess(
    X_exam, 
    [
        cat_Featrues_exam,
        dist_GIT_exam,
        dist_VAT_exam,
        entire_exam
    ]
)

Data preprocessing...


In [10]:
optimized_LGBM = LGBMClassifier(
    task = "predict",
    objective = "binary",
    metric = "auc",
    tree_learner = "data",
    categorical_feature = [0,1,2],
    class_weight={0: 1, 1: 14.291397},
    n_estimators=study.best_params['n_estimators'],
    
    # to deal with overfitting, very important param
    max_depth=study.best_params['max_depth'],
    learning_rate=study.best_params['learning_rate'],
    num_leaves=study.best_params['num_leaves'],
    min_data_in_leaf=study.best_params['min_data_in_leaf'],
    
    #if max_bin becomes small, the accuracy goes up
    max_bin=study.best_params['max_bin'],
    lambda_l1=study.best_params['lambda_l1'],
    lambda_l2=study.best_params['lambda_l2'],
    
    # to deal with overfitting
    min_child_weight=study.best_params['min_child_weight'],
    #for bagging imbalanced
    bagging_fraction=study.best_params['bagging_fraction'],
    pos_bagging_fraction=study.best_params['pos_bagging_fraction'],
    neg_bagging_fraction=study.best_params['neg_bagging_fraction'],
)

In [11]:
print("Executing...")
Y_exam = np.zeros(X_exam_processed.shape[0])
k = 5
kf = StratifiedKFold(n_splits=k)
for tr_index, val_index in kf.split(X_processed,Y_model):
    X_tr,Y_tr = X_processed.iloc[tr_index],Y_model.iloc[tr_index]
    X_val, Y_val = X_processed.iloc[val_index],Y_model.iloc[val_index]
    
    optimized_LGBM.fit(X_tr,Y_tr,eval_metric='auc',eval_set=[(X_tr,Y_tr),(X_val,Y_val)])
    proba = optimized_LGBM.predict_proba(X_exam_processed)[:,1]
    Y_exam = Y_exam + proba
Y_exam = Y_exam/k
thresholds = np.array([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
# the ratio of high prob with different thresholds
for num in thresholds: 
    filtered = Y_exam[np.where(Y_exam>=num)]
    print("the number of probability more than %.2f is %d:" %(num,len(filtered)))
    print("the ratio of probability more than %.2f is : %.4f"%(num, float(len(filtered))/len(Y_exam)))
    print('---------------------------------------------------\n')
# res = pd.DataFrame({'business prob':Y_exam})
# res.to_csv("./part1.csv")


Executing...
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[1]	training's auc: 0.93655	valid_1's auc: 0.933084
[2]	training's auc: 0.93709	valid_1's auc: 0.933696
[3]	training's auc: 0.93738	valid_1's auc: 0.933913
[4]	training's auc: 0.937564	valid_1's auc: 0.934011
[5]	training's auc: 0.93778	valid_1's auc: 0.934275
[6]	training's auc: 0.937872	valid_1's auc: 0.934535
[7]	training's auc: 0.938091	valid_1's auc: 0.934704
[8]	training's auc: 0.938284	valid_1's auc: 0.934859
[9]	training's auc: 0.93835

[3]	training's auc: 0.938324	valid_1's auc: 0.933745
[4]	training's auc: 0.938336	valid_1's auc: 0.933737
[5]	training's auc: 0.938523	valid_1's auc: 0.933783
[6]	training's auc: 0.938695	valid_1's auc: 0.93388
[7]	training's auc: 0.938822	valid_1's auc: 0.934063
[8]	training's auc: 0.938981	valid_1's auc: 0.934142
[9]	training's auc: 0.93917	valid_1's auc: 0.934279
[10]	training's auc: 0.939274	valid_1's auc: 0.934343
[11]	training's auc: 0.93941	valid_1's auc: 0.934413
[12]	training's auc: 0.939492	valid_1's auc: 0.934554
[13]	training's auc: 0.93963	valid_1's auc: 0.934651
[14]	training's auc: 0.939686	valid_1's auc: 0.934739
[15]	training's auc: 0.93972	valid_1's auc: 0.934773
[16]	training's auc: 0.939804	valid_1's auc: 0.934833
[17]	training's auc: 0.939838	valid_1's auc: 0.934878
[18]	training's auc: 0.939939	valid_1's auc: 0.934965
[19]	training's auc: 0.940033	valid_1's auc: 0.934975
[20]	training's auc: 0.940184	valid_1's auc: 0.935097
[21]	training's auc: 0.940258	valid_1's 

[16]	training's auc: 0.939607	valid_1's auc: 0.935419
[17]	training's auc: 0.939695	valid_1's auc: 0.935466
[18]	training's auc: 0.939727	valid_1's auc: 0.935484
[19]	training's auc: 0.939803	valid_1's auc: 0.935517
[20]	training's auc: 0.939862	valid_1's auc: 0.935547
[21]	training's auc: 0.939935	valid_1's auc: 0.935585
[22]	training's auc: 0.94004	valid_1's auc: 0.935657
[23]	training's auc: 0.940102	valid_1's auc: 0.935736
[24]	training's auc: 0.940168	valid_1's auc: 0.935765
[25]	training's auc: 0.940261	valid_1's auc: 0.935839
[26]	training's auc: 0.940341	valid_1's auc: 0.935881
[27]	training's auc: 0.940425	valid_1's auc: 0.935935
[28]	training's auc: 0.940478	valid_1's auc: 0.936007
[29]	training's auc: 0.940528	valid_1's auc: 0.936039
[30]	training's auc: 0.940598	valid_1's auc: 0.936058
[31]	training's auc: 0.940657	valid_1's auc: 0.936093
[32]	training's auc: 0.940724	valid_1's auc: 0.936103
[33]	training's auc: 0.940794	valid_1's auc: 0.936096
[34]	training's auc: 0.94089	

[27]	training's auc: 0.940366	valid_1's auc: 0.935314
[28]	training's auc: 0.940405	valid_1's auc: 0.935344
[29]	training's auc: 0.940467	valid_1's auc: 0.935365
[30]	training's auc: 0.940506	valid_1's auc: 0.935395
[31]	training's auc: 0.940566	valid_1's auc: 0.935453
[32]	training's auc: 0.94061	valid_1's auc: 0.935466
[33]	training's auc: 0.940655	valid_1's auc: 0.935527
[34]	training's auc: 0.940713	valid_1's auc: 0.935538
[35]	training's auc: 0.940797	valid_1's auc: 0.935584
[36]	training's auc: 0.940918	valid_1's auc: 0.935642
[37]	training's auc: 0.941034	valid_1's auc: 0.935693
[38]	training's auc: 0.941125	valid_1's auc: 0.935743
[39]	training's auc: 0.941208	valid_1's auc: 0.935759
[40]	training's auc: 0.941251	valid_1's auc: 0.935771
[41]	training's auc: 0.941311	valid_1's auc: 0.935817
[42]	training's auc: 0.94134	valid_1's auc: 0.935816
[43]	training's auc: 0.94139	valid_1's auc: 0.935827
[44]	training's auc: 0.94144	valid_1's auc: 0.935839
[45]	training's auc: 0.941462	va

[38]	training's auc: 0.940794	valid_1's auc: 0.938112
[39]	training's auc: 0.940872	valid_1's auc: 0.938106
[40]	training's auc: 0.940954	valid_1's auc: 0.938129
[41]	training's auc: 0.940996	valid_1's auc: 0.938139
[42]	training's auc: 0.941018	valid_1's auc: 0.938143
[43]	training's auc: 0.941056	valid_1's auc: 0.938152
[44]	training's auc: 0.941094	valid_1's auc: 0.938162
[45]	training's auc: 0.941149	valid_1's auc: 0.938181
[46]	training's auc: 0.941185	valid_1's auc: 0.938183
[47]	training's auc: 0.941236	valid_1's auc: 0.938199
[48]	training's auc: 0.941267	valid_1's auc: 0.938212
[49]	training's auc: 0.941321	valid_1's auc: 0.938239
[50]	training's auc: 0.94135	valid_1's auc: 0.938238
[51]	training's auc: 0.941377	valid_1's auc: 0.938235
[52]	training's auc: 0.941426	valid_1's auc: 0.938269
[53]	training's auc: 0.941466	valid_1's auc: 0.938289
[54]	training's auc: 0.941592	valid_1's auc: 0.938296
[55]	training's auc: 0.941634	valid_1's auc: 0.93831
[56]	training's auc: 0.94171	v

In [12]:
from sklearn.metrics import accuracy_score
Y_exam = np.zeros(X_exam_processed.shape[0])

accuracy_Arr = np.array([])
popup_Arr = np.array([])
survey_Arr = np.array([])

k = 5
kf = StratifiedKFold(n_splits=k)
thresholds = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for th in thresholds:
    popup_P = 0
    survey_P = 0
    accuracy = 0
    for tr_index, val_index in kf.split(X_processed,Y_model):
        X_tr,Y_tr = X_processed.iloc[tr_index],Y_model.iloc[tr_index]
        X_val, Y_val = X_processed.iloc[val_index],Y_model.iloc[val_index]

        optimized_LGBM.fit(X_tr,Y_tr,eval_metric='auc')
        # Generate the columns
        proba = optimized_LGBM.predict_proba(X_val)[:,1]
        pred = pd.DataFrame({'business prob_pred':proba})
        popup = pd.DataFrame({'popup':np.zeros(X_val.shape[0])})
        survey = pd.DataFrame({'survey':np.zeros(X_val.shape[0])})
        # Merge
        res = pd.concat([pred,popup],axis=1)
        res = pd.concat([res,survey],axis=1)
        
        # If predicted proba is more than th, put his popup as 1
        res.loc[res['business prob_pred'] >= th, 'popup'] = 1
        res.loc[res['business prob_pred'] >= th, 'survey'] = 1
        
        accuracy += accuracy_score(Y_val, res['popup'])
        res = pd.concat([res,Y_val.reset_index(drop=True)],axis=1) # Merge res with Y_val
        popup_P += res.loc[res['popup']==1,'business'].sum()*(500000*0.01) - res['popup'].sum()*400
        survey_P += res.loc[res['survey']==1,'business'].sum()*(500000*0.036) - res['survey'].sum()*5000
        
    accuracy_Arr = np.append(accuracy_Arr,accuracy/k)
    popup_Arr = np.append(popup_Arr,popup_P/k)
    survey_Arr = np.append(survey_Arr,survey_P/k)
    

#     print('---------------------------------')
#     print(accuracy_score(Y_val, res['popup']))
#     print('---------------------------------')
#     print('net profit:')
#     profit = res.loc[res['popup']==1,'business'].sum()*(500000*0.01) - res['popup'].sum()*400
#     print(profit)
#     print('---------------------------------')


[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.13149839911037

[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.13149839911037

[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.13149839911037

[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.1314983991103761, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.1314983991103761
[LightGBM] [Warning] min_data_in_leaf is set=359, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=359
[LightGBM] [Warning] lambda_l1 is set=0.8905551242240669, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.8905551242240669
[LightGBM] [Warning] bagging_fraction is set=0.21312852654316028, subsample=1.0 will be ignored. Current value: bagging_fraction=0.21312852654316028
[LightGBM] [Warning] lambda_l2 is set=0.13149839911037

In [13]:
df = pd.DataFrame({'threshold':thresholds, 'avg_Accuracy':accuracy_Arr, 'avg_Popup_Profit':popup_Arr, 'avg_Survey_Porfit':survey_Arr})
df.set_index('threshold')
print(df)

   threshold  avg_Accuracy  avg_Popup_Profit  avg_Survey_Porfit
0        0.0      0.068565        -7730920.0       -509178200.0
1        0.1      0.663684        24146360.0       -107437000.0
2        0.2      0.704705        25991120.0        -80417000.0
3        0.3      0.735398        27237000.0        -60455800.0
4        0.4      0.766618        28197200.0        -40737000.0
5        0.5      0.800233        28735120.0        -20449400.0
6        0.6      0.835155        28591040.0          -712400.0
7        0.7      0.890824        26255240.0         26738600.0
8        0.8      0.955558        19782120.0         51503200.0
9        0.9      0.960698        18460760.0         51931800.0
